## SiFID evaluation metric


[Official SinGAN Repository](https://github.com/tamarott/SinGAN)


- In order to compute the SIFID evaluation metric, the synthetic images and the original one need to be saved in the right format ()JPEG, squared and .jpg extension
- The output is a .txt file named 'metrics_sifid_scores.txt' with the id of the training image and the mean and standard deviation of the SiFID metric.


In [1]:
import os
from PIL import Image

In [2]:
root=input("Path of SinGAN folder: ")
num_samples=input("Number of samples generated with each SinGAN model: ")
num_samples=int(num_samples)


# List of training images of the model generated samples you want to evaluate  
inputs = ['resized_2180.png', 'resized_168.png', 'resized_3884.png', 'resized_51.png']

# Each of the generated samples needs to be inside different folders and in a specific format. 
sifid_real_path=os.path.join(root, r'sifid_data/real')
if not os.path.exists(sifid_real_path):
    os.makedirs(sifid_real_path, exist_ok=True)

sifid_fake_path=os.path.join(root, r'sifid_data/fake')
if not os.path.exists(sifid_fake_path):
    os.makedirs(sifid_fake_path, exist_ok=True)

# Take the input image and move it to a folder named 'sifid_data/real'
for input_img in inputs:
    real_img_path = os.path.join(os.path.join(root,'Input/Images'), input_img)
    num_img = input_img.split('_')[1]
    num_img = num_img.split('.png')[0]
    real_img_path2=os.path.join(sifid_real_path, f"{num_img}")
    os.makedirs(real_img_path2, exist_ok=True)
    #convert the real image to JPEG format and copy to the sifid_data folder
    with Image.open(real_img_path) as img:
        img = img.convert('RGB')
        img.save(os.path.join(real_img_path2, f'{num_img}.jpg'), format='JPEG')

    #select each one of the fake images, resize it, convert it into JPEG and save it in 'sifid_data/fake' folder
    fake_path=os.path.join(root, os.path.join('synthetic_images\zoom3', num_img))
    sifid_fake_path2=os.path.join(sifid_fake_path, f"{num_img}") #eg '/home/marta/SinGAN/sifid_data/fake/2789'
    for i in range(1, 70):
        fake_selected = f"{i}.png" #e.g. '50.png'
        fake_file=os.path.join(fake_path, fake_selected)
        # Create a new directory for the fake image
        fake_dir = os.path.join(sifid_fake_path2, str(i))
        os.makedirs(fake_dir, exist_ok=True)
        with Image.open(fake_file) as img:
            img = img.convert('RGB')
            img = img.resize((224, 224))
            # Save the image as a JPEG file in the output folder with quality level 90
            output_path = os.path.join(fake_dir, f'{i}.jpg')
            img.save(output_path, format='JPEG', quality=90)

Create a noise image to compute an upperbound for SiFID:

In [4]:
from PIL import Image
import statistics
import numpy as np
import os
noise_images_path=os.path.join(root, r'sifid_data/noise_images')

for i in range(1, 100):
    # # Create a 224x224 numpy array of random values between 0 and 255
    noise = np.random.randint(0, 256, (224, 224), dtype=np.uint8)
    noise = Image.fromarray(noise)
    image_name = f"{i}.jpg"
    # Create a new directory for the fake image
    noise_file_path = os.path.join(noise_images_path, str(i))
    os.makedirs(noise_file_path, exist_ok=True)
    noise = noise.convert('RGB')    
    # Save the image as a JPEG file in the output folder with quality level 90
    noise.save(os.path.join(noise_file_path, image_name), format='JPEG')



In [8]:
from PIL import Image
import statistics
import numpy as np
import os
import statistics
input_img = '3884_resized.png'
num_img = input_img.split('_')[0] 
num_img = num_img.split('.png')[0]

root_real=os.path.join(root, r'sifid_data/real')
command_real_path=os.path.join(root_real, str(num_img))
root_fake=os.path.join(root, r'sifid_data/fake')
for i in range(num_samples-1):
    command_fake_path=os.path.join(os.path.join(root_fake, str(num_img)), str(i+1)) #if it does not work do with:command_real_path=f'C:\\Users\\marta\\Documents\\MASTER_UB\\TFM\\SinGAN\\sifid_data\\real\\{num_img}'
    !python SIFID/sifid_score.py --path2real $command_real_path --path2fake $command_fake_path

    with open('sifid_scores.txt', 'r') as f:
        content = f.read()
#sifid scores
nums = [float(x) for x in content.split()]

#delete the file for the next original image:
filename = 'sifid_scores.txt'
if os.path.exists(filename):
    os.remove(filename)

#mean and standard deviation
mean = np.mean(nums)
std_dev = statistics.stdev(nums)
print(num_img, ' : ', str(mean), '+/-', str(std_dev), '\n')
filename = 'metrics_sifid_scores.txt'

# Open the file for appending (which creates the file if it doesn't exist)
with open(filename, 'a') as f:
# Write the sifid score to the file
    f.write(str(num_img)+' : '+ str(mean)+ '+/-'+str(std_dev)+ '\n')


SIFID:  0.10560187


c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 53.79it/s]

100%|██████████| 1/1 [00:00<00:00, 111.13it/s]


SIFID:  0.09285955


c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 58.82it/s]

100%|██████████| 1/1 [00:00<00:00, 111.12it/s]


SIFID:  0.15690933


...

c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 47.70it/s]

100%|██████████| 1/1 [00:00<00:00, 105.12it/s]


SIFID:  0.10759497


c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 47.61it/s]

100%|██████████| 1/1 [00:00<00:00, 125.02it/s]


SIFID:  0.1625672
3884  :  0.1432699998985507 +/- 0.035362259532227595 



c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 41.60it/s]

100%|██████████| 1/1 [00:00<00:00, 117.42it/s]


In [ ]:
# img = Image.open('/content/gdrive/MyDrive/TFM/data/2789_real/2789.jpg')
# rotated_image = img.rotate(180)
# rotate_path='/content/gdrive/MyDrive/TFM/data/rotated_image'
# if not os.path.exists(rotate_path):
#     os.makedirs(rotate_path)
# rotated_image.save(os.path.join(rotate_path, '2789.jpg'), format='JPEG')


I modify the sifid_score.py script, so that when the following command is running, the sifid scores are written in a .txt output file for each of the fake samples, so then the mean and standar deviation can be obtained

In [11]:
import numpy as np
import statistics
import os

num_samples=100
# inputs = ['2789_resized.png', '3941_resized.png', '77_resized.png', '13_resized.png']
inputs = ['77_resized.png']
for input_img in inputs:
    num_img = input_img.split('_')[0] 
    command_real_path=f'C:\\Users\\marta\\Documents\\MASTER_UB\\TFM\\SinGAN\\sifid_data\\real\\{num_img}'
    for i in range(num_samples-1):
        command_fake_path=f'C:\\Users\\marta\\Documents\\MASTER_UB\\TFM\\SinGAN\\sifid_data\\fake\\{num_img}\\{i+1}'
        !python SIFID/sifid_score.py --path2real $command_real_path --path2fake $command_fake_path

    with open('sifid_scores.txt', 'r') as f:
        content = f.read()
    #sifid scores
    nums = [float(x) for x in content.split()]

    #delete the file for the next original image:
    filename = 'sifid_scores.txt'
    if os.path.exists(filename):
     os.remove(filename)

    #mean and standard deviation
    mean = np.mean(nums)
    std_dev = statistics.stdev(nums)
    print(num_img, ' : ', str(mean), '+/-', str(std_dev), '\n')
    filename = 'metrics_sifid_scores.txt'

    # Open the file for appending (which creates the file if it doesn't exist)
    with open(filename, 'a') as f:
    # Write the sifid score to the file
        f.write(num_img+ ' : '+ str(mean)+ '+/-'+str(std_dev)+ '\n')

SIFID:  0.49070105


c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 21.28it/s]

100%|██████████| 1/1 [00:00<00:00, 99.97it/s]


SIFID:  0.48895642


c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 24.39it/s]

100%|██████████| 1/1 [00:00<00:00, 99.96it/s]


SIFID:  0.49039593


...

c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 21.74it/s]

100%|██████████| 1/1 [00:00<00:00, 95.16it/s]


SIFID:  0.4861153


c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 22.22it/s]

100%|██████████| 1/1 [00:00<00:00, 99.99it/s]


SIFID:  0.49327162
77  :  0.4958152615151516 +/- 0.006036743577115872 



c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marta\Anaconda\envs\TFMmarta\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

100%|██████████| 1/1 [00:00<00:00, 22.73it/s]

100%|██████████| 1/1 [00:00<00:00, 90.81it/s]



    # Check if the file exists
    if not os.path.exists(filename):
    # If the file doesn't exist, create it
    with open(filename, 'w') as f:
        pass  #just create the file